In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(link)

In [3]:
table[0].head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


### 'For' loop to rewrite every 'not assigned' into a np.nan

In [4]:
for column in table[0].columns:
    for i in range(table[0].shape[0]):
        if 'assigned' in table[0][column].iloc[i]:
            table[0][column].iloc[i] = np.nan

In [5]:
table[0].head()

,0,1,2,3,4,5,6,7,8
0,NaN,NaN,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,NaN,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),NaN,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),NaN,NaN,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,NaN,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),NaN,NaN,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,NaN,NaN,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),NaN,NaN,NaN
4,M1GScarborough(Woburn),NaN,NaN,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),NaN,NaN,NaN


### 'For' loop to extract from the table the values of postal code, borough and neighborhood

In [6]:
postal_code = []
borough = []
neighborhood = []
for column in table[0].columns:
    postal_code.append(table[0][column].apply(lambda x: str(x)[0:3]))
    borough.append(table[0][column].apply(lambda x: str(x)[3:].split('(')[0]))
    for neigh in (table[0][column].apply(lambda x: str(x)[3:].split('('))):
                  if len(neigh) > 1:
                       neighborhood.append(neigh[1][:-1])
                  else:
                        neighborhood.append(neigh[0])

In [7]:
#'For' loop to get rid of NaN values in the list of lists postal_code
PostalCode = []
for lista in postal_code:
    for code in lista:
        if code != 'nan':
            PostalCode.append(code)

In [8]:
#'For' loop to get rid of empty values (len(value) == 0) in the list of lists borough
Borough = []
for lista in borough:
    for b in lista:
        if len(b) > 0:
            Borough.append(b)

In [9]:
#'For' loop to get rid of empty values (len(value) == 0) in the list neighborhood
Neighborhood = []
for i in neighborhood:
    if len(i) > 0:
        Neighborhood.append(i)

In [10]:
d = {'PostalCode':PostalCode, 'Borough':Borough, 'Neighborhood':Neighborhood}
df = pd.DataFrame(d)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Retrieving the latitude and longitude coordinates of each neighborhood

In [11]:
lat_lng_df = pd.read_csv('http://cocl.us/Geospatial_data')
lat_lng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#As state below, the postal code column of the latitude and longitude dataframe is the same as the df's postal code column,
#so we can just add the columns of interest into the df
sum(lat_lng_df['Postal Code'] == df['PostalCode'])

103

In [13]:
df['Latitude'] = lat_lng_df['Latitude']
df['Longitude'] = lat_lng_df['Longitude']

In [14]:
#The final Data Frame
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
df.shape

(103, 5)

## Explore and clustering the neighborhoods in Toronto

In [16]:
#Extract the indexs of Toronto's boroughs
indexes = []
for i, word in enumerate(df['Borough']):
    if 'Toronto' in word.split(' '):
        indexes.append(i)      

In [17]:
#Building the Toronto dataframe
toronto_df = df.iloc[indexes].reset_index().drop('index', axis=1)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
3,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923


### Use geopy library to get the latitude and longitude values of Toronto

In [18]:
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium
import requests

In [19]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent='to_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207


### Create a map of Toronto with neighborhoods

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#add the markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_toronto)
    
map_toronto


### Exploring with Foursquare API

In [21]:
CLIENT_ID = 'UP1DGQPQB5AKCJI5KSHPZYNPKTV0SAKMW22T4YPGWGSIXACB'
CLIENT_SECRET = 'TQLQWRSSUZQW44ZEO4CB1X2U2HORNLLIB0F102O30YDYEFQY'
VERSION = '20180605'

#### Exploring the first neighborhood of toronto_df

In [22]:
neighborhood_name = toronto_df.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_df.loc[0, 'Latitude']
neighborhood_longitude = toronto_df.loc[0, 'Longitude']

#### Now we get the top 100 venues that are in the first neighborhood within a radius of 500 meters

In [23]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, 500)
url

'https://api.foursquare.com/v2/venues/explore?client_id=UP1DGQPQB5AKCJI5KSHPZYNPKTV0SAKMW22T4YPGWGSIXACB&client_secret=TQLQWRSSUZQW44ZEO4CB1X2U2HORNLLIB0F102O30YDYEFQY&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7cbc7c14a126001b1cb973'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [25]:
#Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Cleaning the json to build a pandas dataframe

In [26]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


### Function to repeat the same process to all neighborhoods in Toronto

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(name, 
                             lat, 
                            lng, 
                            v['venue']['name'], 
                            v['venue']['location']['lat'], 
                            v['venue']['location']['lng'], 
                            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                            'Neighborhood Latitude', 
                            'Neighborhood Longitude', 
                            'Venue', 
                            'Venue Latitude', 
                            'Venue Longitude', 
                            'Venue Category']
    return nearby_venues

In [28]:
LIMIT=100
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'], latitudes=toronto_df['Latitude'], longitudes=toronto_df['Longitude'])

The Beaches
The Danforth East
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(1571, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,54,54,54,54,54,54
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,81,81,81,81,81,81
Christie,18,18,18,18,18,18
Church and Wellesley,88,88,88,88,88,88
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,37,37,37,37,37,37
Davisville North,7,7,7,7,7,7


### Analyze each neighborhood

In [31]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood']

#Move neighborhood to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
toronto_onehot.shape

(1571, 231)

### Now we group rows by neighborhoods and take the mean of the frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Central Bay Street,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.012346,0.000000,0.00,...,0.00000,0.00,0.012346,0.000000,0.000000,0.012346,0.000000,0.000000,0.00,0.012346
4,Christie,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,Church and Wellesley,0.011364,0.0000,0.0000,0.000,0.000,0.000,0.011364,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.011364,0.000000,0.011364,0.011364,0.00,0.022727
6,Commerce Court / Victoria Hotel,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,...,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000
7,Davisville,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,Davisville North,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Dufferin / Dovercourt Village,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.00,0.000000


In [34]:
toronto_grouped.shape

(37, 231)

### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhoods']:
    print('----'+hood+'----')
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype('float')
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1     Cheese Shop  0.04
2          Bakery  0.04
3      Restaurant  0.04
4  Farmers Market  0.04


----Brockton / Parkdale Village / Exhibition Place----
                 venue  freq
0       Breakfast Spot  0.09
1          Coffee Shop  0.09
2            Nightclub  0.09
3                 Café  0.09
4  Japanese Restaurant  0.04


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4          Boutique  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.05
2        Burger Joint  0.04
3      Sandwich Place  0.04
4      Ice Cream Shop  0.04


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11
3     Restaurant  

### We will put that into a pandas dataframe

In [36]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [115]:
#Now let's create the dataframe
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

#create the columns labels
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    if ind < 3:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    else:
        columns.append('{}th Most Common Venue'.format(ind+1))

        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for indi in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[indi, 1:] = return_most_common_venues(toronto_grouped.iloc[indi, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Cheese Shop,Restaurant,Café,Pub
1,Brockton / Parkdale Village / Exhibition Place,Coffee Shop,Breakfast Spot,Café,Nightclub,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Bar,Rental Car Location,Harbor / Marina,Sculpture Garden
3,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Salad Place,Bubble Tea Shop,Café,Chinese Restaurant
4,Christie,Grocery Store,Café,Park,Candy Store,Athletics & Sports,Diner,Italian Restaurant,Restaurant,Gas Station,Baby Store


# Cluster Neighborhoods

In [38]:
from sklearn.cluster import KMeans

In [116]:
k_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', axis=1)
kmeans = KMeans(n_clusters=k_clusters, random_state=0)
kmeans.fit(toronto_grouped_clustering)
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1,
       1, 1, 0, 3, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [108]:
toronto_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

In [117]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_df.drop('PostalCode', axis=1).join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Coffee Shop,Pub,Health Food Store,Neighborhood,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,0,Park,Coffee Shop,Convenience Store,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Frozen Yogurt Shop,Pub,Pizza Place
3,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,1,Park,Pizza Place,Pub,Ice Cream Shop,Fish & Chips Shop,Brewery,Steakhouse,Fast Food Restaurant,Restaurant,Italian Restaurant
4,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Brewery,American Restaurant,Yoga Studio,Neighborhood,Seafood Restaurant,Sandwich Place,Cheese Shop


### Let's visualize the clusters

In [96]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [118]:
map_clusters = folium.Map([latitude, longitude], zoom_start=11)

#Color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], 
                       radius=5, 
                       popup=label, 
                       color=rainbow[cluster-1], 
                       fill=True, 
                       fill_color=rainbow[cluster-1], 
                       fill_opacity=0.7).add_to(map_clusters)

map_clusters    

# Examine the clusters

### Cluster 0

In [103]:
toronto_merged[toronto_merged['Cluster Labels'] == 0]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,0,Park,Coffee Shop,Convenience Store,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
6,Central Toronto,Davisville North,43.712751,-79.390197,0,Department Store,Park,Breakfast Spot,Convenience Store,Sandwich Place,Hotel,Food & Drink Shop,Diner,Electronics Store,Eastern European Restaurant
11,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Yoga Studio,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
24,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Park,Jewelry Store,Trail,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 1

In [119]:
toronto_merged[toronto_merged['Cluster Labels'] == 1]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Coffee Shop,Pub,Health Food Store,Neighborhood,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Frozen Yogurt Shop,Pub,Pizza Place
3,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,1,Park,Pizza Place,Pub,Ice Cream Shop,Fish & Chips Shop,Brewery,Steakhouse,Fast Food Restaurant,Restaurant,Italian Restaurant
4,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Brewery,American Restaurant,Yoga Studio,Neighborhood,Seafood Restaurant,Sandwich Place,Cheese Shop
7,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Italian Restaurant,Restaurant,Dessert Shop,Café,Mexican Restaurant,Fast Food Restaurant,Sporting Goods Shop
8,Central Toronto,Davisville,43.704324,-79.388790,1,Dessert Shop,Sandwich Place,Café,Thai Restaurant,Italian Restaurant,Coffee Shop,Gym,Sushi Restaurant,Pizza Place,Asian Restaurant
10,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049,1,Pub,Coffee Shop,Bank,Light Rail Station,Vietnamese Restaurant,Supermarket,Liquor Store,American Restaurant,Pizza Place,Sushi Restaurant
12,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Pub,Chinese Restaurant,Bakery,Pizza Place,Grocery Store,Pet Store
13,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Yoga Studio,Café,Pub,Hotel,Gastropub
14,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Mexican Restaurant,Theater,Café,Shoe Store,Breakfast Spot,Restaurant


### Cluster 2

In [120]:
toronto_merged[toronto_merged['Cluster Labels'] == 2]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Central Toronto,Moore Park / Summerhill East,43.689574,-79.38316,2,Playground,Yoga Studio,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 3

In [121]:
toronto_merged[toronto_merged['Cluster Labels'] == 3]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,Roselawn,43.711695,-79.416936,3,Garden,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 4

In [122]:
toronto_merged[toronto_merged['Cluster Labels'] == 4]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,Lawrence Park,43.72802,-79.38879,4,Park,Swim School,Bus Line,Yoga Studio,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
